In [ ]:
!pip install wandb

!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

!tar -xf dakshina_dataset_v1.0.tar

     |████████████████████████████████| 1.8 MB 27.3 MB/s 
     |████████████████████████████████| 181 kB 67.6 MB/s 
     |████████████████████████████████| 145 kB 66.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=da0a5549ef1211e01c45bc06fdff6e50dae278a696ba3fa8757bc1f5997e34ae
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
--2022-05-12 10:03:14--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[=========

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib

DATAPATH = "./dakshina_dataset_v1.0"

trainpath = os.path.join(DATAPATH, "hi", "lexicons", "hi"+".translit.sampled.train.tsv")
valpath = os.path.join(DATAPATH, "hi", "lexicons", "hi"+".translit.sampled.dev.tsv")
testpath = os.path.join(DATAPATH, "hi", "lexicons", "hi"+".translit.sampled.test.tsv")
train = pd.read_csv(
    trainpath,
    sep="\t",
    names=["tgt", "src", "count"],
)
val = pd.read_csv(
    valpath,
    sep="\t",
    names=["tgt", "src", "count"],
)
test = pd.read_csv(
    testpath,
    sep="\t",
    names=["tgt", "src", "count"],
)



# create train data
#train_data = preprocess(list(train["src"]), list(train["tgt"]))

In [ ]:
source=list(train["src"])
target=list(train["tgt"])

source_chars = set()
target_chars = set()

#removing non str nan types
source = [str(x) for x in source]
target = [str(x) for x in target]

source_words = []
target_words = []

for src, tgt in zip(source, target):
    tgt = "\t" + tgt + "\n"
    source_words.append(src)
    target_words.append(tgt)
    for char in src:
        if char not in source_chars:
            source_chars.add(char)
    for char in tgt:
        if char not in target_chars:
            target_chars.add(char)

source_chars = sorted(list(source_chars))
target_chars = sorted(list(target_chars))

#The space needs to be appended so that the encode function doesn't throw errors
source_chars.append(" ")
target_chars.append(" ")

num_encoder_tokens = len(source_chars)
num_decoder_tokens = len(target_chars)
max_source_length = max([len(txt) for txt in source_words])
max_target_length = max([len(txt) for txt in target_words])

print("Number of samples:", len(source))
print("Source Vocab length:", num_encoder_tokens)
print("Target Vocab length:", num_decoder_tokens)
print("Max sequence length for inputs:", max_source_length)
print("Max sequence length for outputs:", max_target_length)


Number of samples: 44204
Source Vocab length: 27
Target Vocab length: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [ ]:
def dictionary_lookup( vocab):
  char2int = dict([(char, i) for i, char in enumerate(vocab)])
  int2char = dict((i, char) for char, i in char2int.items())
  return char2int, int2char


In [ ]:
def encode(source, target, source_chars, target_chars, source_char2int=None, target_char2int=None):
        num_encoder_tokens = len(source_chars)
        num_decoder_tokens = len(target_chars)
        max_source_length = max([len(txt) for txt in source])
        max_target_length = max([len(txt) for txt in target])

        source_vocab, target_vocab = None, None
        if source_char2int == None and target_char2int == None:
            print("Generating the dictionary lookups for character to integer mapping and back")
            source_char2int, source_int2char = dictionary_lookup(source_chars)
            target_char2int, target_int2char = dictionary_lookup(target_chars)

            source_vocab = (source_char2int, source_int2char)
            target_vocab = (target_char2int, target_int2char)

        encoder_input_data = np.zeros(
            (len(source), max_source_length, num_encoder_tokens), dtype="float32"
        )
        decoder_input_data = np.zeros(
            (len(source), max_target_length, num_decoder_tokens), dtype="float32"
        )
        decoder_target_data = np.zeros(
            (len(source), max_target_length, num_decoder_tokens), dtype="float32"
        )

        for i, (input_text, target_text) in enumerate(zip(source, target)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t, source_char2int[char]] = 1.0
            encoder_input_data[i, t + 1 :, source_char2int[" "]] = 1.0
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, target_char2int[char]] = 1.0
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, target_char2int[char]] = 1.0
            decoder_input_data[i, t + 1 :, target_char2int[" "]] = 1.0
            decoder_target_data[i, t:, target_char2int[" "]] = 1.0
        if source_vocab != None and target_vocab != None:
            return (
                encoder_input_data,
                decoder_input_data,
                decoder_target_data,
                source_vocab,
                target_vocab,
            )
        else:
            return encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
train_data=encode(source_words, target_words, source_chars, target_chars)

Generating the dictionary lookups for character to integer mapping and back


In [ ]:
 # create train data
(train_encoder_input,
    train_decoder_input,
    train_decoder_target,
    source_vocab,
    target_vocab,
) = train_data
source_char2int, source_int2char = source_vocab
target_char2int, target_int2char = target_vocab


In [ ]:
 # create val data (only encode function suffices as the dictionary lookup should be kep the same.
val_data = encode(
    val["src"].to_list(),
    val["tgt"].to_list(),
    list(source_char2int.keys()),
    list(target_char2int.keys()),
    source_char2int=source_char2int,
    target_char2int=target_char2int,
)
val_encoder_input, val_decoder_input, val_decoder_target = val_data
source_char2int, source_int2char = source_vocab
target_char2int, target_int2char = target_vocab

# create test data
test_data = encode(
    test["src"].to_list(),
    test["tgt"].to_list(),
    list(source_char2int.keys()),
    list(target_char2int.keys()),
    source_char2int=source_char2int,
    target_char2int=target_char2int,
)
test_encoder_input, test_decoder_input, test_decoder_target = test_data
source_char2int, source_int2char = source_vocab
target_char2int, target_int2char = target_vocab


In [ ]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
 

#from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Activation, LSTM, SimpleRNN, GRU, TimeDistributed
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model, Sequential,  Model
from tensorflow.keras.callbacks import EarlyStopping


import wandb


class S2STranslation():

    def __init__(self, numEncoders,cell_type,latentDim,dropout,numDecoders,hidden,srcChar2Int, tgtChar2Int, using_pretrained_model = False):
        
        self.numEncoders = numEncoders
        self.cell_type = cell_type
        self.latentDim = latentDim
        self.dropout =dropout
        self.numDecoders = numDecoders
        self.hidden = hidden
        self.tgtChar2Int = tgtChar2Int
        self.srcChar2Int = srcChar2Int

    def build_configurable_model(self):       
        if self.cell_type == "RNN":
            # encoder
            encoder_inputs = Input(shape=(None, len(self.srcChar2Int)))
            encoder_outputs = encoder_inputs
            for i in range(1, self.numEncoders + 1):
                encoder = SimpleRNN(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                encoder_outputs, state = encoder(encoder_inputs)
            encoder_states = [state]

            # decoder
            decoder_inputs = Input(shape=(None, len(self.tgtChar2Int)))
            decoder_outputs = decoder_inputs
            for i in range(1, self.numDecoders + 1):
                decoder = SimpleRNN(
                    self.latentDim,
                    return_sequences=True,
                    return_state=True,
                    dropout=self.dropout,
                )
                decoder_outputs, _ = decoder(decoder_inputs, initial_state=encoder_states)

            # dense
            hidden = Dense(self.hidden, activation="relu")
            hidden_outputs = hidden(decoder_outputs)
            decoder_dense = Dense(len(self.tgtChar2Int), activation="softmax")
            decoder_outputs = decoder_dense(hidden_outputs)
            model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
            return model
        
        elif self.cell_type == "LSTM":
            # encoder
            encoder_inputs = Input(shape=(None, len(self.srcChar2Int)))
            encoder_outputs = encoder_inputs
            for i in range(1, self.numEncoders + 1):
                encoder = LSTM(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                encoder_outputs, state_h, state_c = encoder(encoder_outputs)
            encoder_states = [state_h, state_c]

            # decoder
            decoder_inputs = Input(shape=(None, len(self.tgtChar2Int)))
            decoder_outputs = decoder_inputs
            for i in range(1, self.numDecoders + 1):
                decoder = LSTM(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                decoder_outputs, _, _ = decoder(
                    decoder_outputs, initial_state=encoder_states
                )

            # dense
            hidden = Dense(self.hidden, activation="relu")
            hidden_outputs = hidden(decoder_outputs)
            decoder_dense = Dense(len(self.tgtChar2Int), activation="softmax")
            decoder_outputs = decoder_dense(hidden_outputs)
            model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
            return model
        
        elif self.cell_type == "GRU":
            # encoder
            encoder_inputs = Input(shape=(None, len(self.srcChar2Int)))
            encoder_outputs = encoder_inputs
            for i in range(1, self.numEncoders + 1):
                encoder = GRU(
                    self.latentDim,
                    return_state=True,
                    return_sequences=True,
                    dropout=self.dropout,
                )
                encoder_outputs, state = encoder(encoder_inputs)
            encoder_states = [state]

            # decoder
            decoder_inputs = Input(shape=(None, len(self.tgtChar2Int)))
            decoder_outputs = decoder_inputs
            for i in range(1, self.numDecoders + 1):
                decoder = GRU(
                    self.latentDim,
                    return_sequences=True,
                    return_state=True,
                    dropout=self.dropout,
                )
                decoder_outputs, _ = decoder(decoder_inputs, initial_state=encoder_states)

            # dense
            hidden = Dense(self.hidden, activation="relu")
            hidden_outputs = hidden(decoder_outputs)
            decoder_dense = Dense(len(self.tgtChar2Int), activation="softmax")
            decoder_outputs = decoder_dense(hidden_outputs)
            model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
            return model

In [ ]:
#sweep config
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes"
  }
metric_dict={
  "name": "val_accuracy",
  "goal": "maximize"
  }
sweep_config['metric']=metric_dict
parameters_dict= {
        
        "cell_type": {"values": ["RNN", "GRU", "LSTM"]},
        
        "latentDim": {"values": [256]},
        
        "hidden": {"values": [128, 64]},
        
        "optimiser": {"values": ["adam"]},
        
        "numEncoders": {"values": [1, 2, 3]},
        
        "numDecoders": {"values": [1, 2, 3]},
        
        "dropout": {"values": [0.2, 0.3]},
        
        "epochs": {"values": [5,10,15, 20]},
        
        "batch_size": {"values": [32, 64]},
    }

sweep_config['parameters'] = parameters_dict


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-Assignment-3")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: ihtplb13
Sweep URL: https://wandb.ai/muk465/CS6910-Assignment-3/sweeps/ihtplb13


In [ ]:
import numpy as np
import pandas as pd
import os

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import RNN, LSTM, GRU, Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

import wandb
from wandb.keras import WandbCallback


import tensorflow as tf


def train():

  config_defaults = {
      "cell_type": "RNN",
      "latentDim": 256,
      "hidden": 128,
      "optimiser": "adam",
      "numEncoders": 1,
      "numDecoders": 1,
      "dropout": 0.2,
      "epochs": 1,
      "batch_size": 64,
  }


  wandb.init(config=config_defaults,  project="CS6910-Assignment-3")
  config = wandb.config
  wandb.run.name = (
      str(config.cell_type)
      + "eng"
      + str(config.numEncoders)
      + "_"
      + "hi"
      + "_"
      + str(config.numDecoders)
      + "_"
      + config.optimiser
      + "_"
      + str(config.epochs)
      + "_"
      + str(config.dropout) 
      + "_"
      + str(config.batch_size)
      + "_"
      + str(config.latentDim)
  )
  wandb.run.save()

  modelInit = S2STranslation(
    config.numEncoders, 
    config.cell_type, 
    config.latentDim, 
    config.dropout, 
    config.numDecoders, 
    config.hidden ,
    srcChar2Int=source_char2int, 
    tgtChar2Int=target_char2int
    )

  model = modelInit.build_configurable_model()

  model.summary()

  model.compile(
      optimizer=config.optimiser,
      loss="categorical_crossentropy",
      metrics=["accuracy"],
  )

  earlystopping = EarlyStopping(
      monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto"
  )

  model.fit(
      [train_encoder_input, train_decoder_input],
      train_decoder_target,
      batch_size=config.batch_size,
      epochs=config.epochs,
      validation_data=([val_encoder_input, val_decoder_input], val_decoder_target),
      callbacks=[earlystopping, WandbCallback()],
  )

  model.save(os.path.join("./TrainedModels", wandb.run.name))    
  wandb.finish()






In [ ]:
wandb.agent(sweep_id, train, count = 200)

wandb: Agent Starting Run: 9h89d7ed with config:
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout: 0.2
wandb: 	epochs: 20
wandb: 	hidden: 64
wandb: 	latentDim: 256
wandb: 	numDecoders: 1
wandb: 	numEncoders: 2
wandb: 	optimiser: adam
wandb: Currently logged in as: muk465. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 27)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 66)]   0           []                               
                                                                                                  
 gru_1 (GRU)                    [(None, None, 256),  218880      ['input_1[0][0]']                
                                 (None, 256)]                                                     
                                                                                                  
 gru_2 (GRU)                    [(None, None, 256),  248832      ['input_2[0][0]',            

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20
691/691 [==============================] - 14s 10ms/step - loss: 1.0691 - accuracy: 0.7275 - val_loss: 2.1386 - val_accuracy: 0.6106 - _timestamp: 1652349873.0000 - _runtime: 26.0000
Epoch 2/20
691/691 [==============================] - 6s 9ms/step - loss: 0.8996 - accuracy: 0.7531 - val_loss: 2.0793 - val_accuracy: 0.6420 - _timestamp: 1652349879.0000 - _runtime: 32.0000
Epoch 3/20
691/691 [==============================] - 6s 9ms/step - loss: 0.7801 - accuracy: 0.7824 - val_loss: 2.2180 - val_accuracy: 0.6569 - _timestamp: 1652349885.0000 - _runtime: 38.0000
Epoch 4/20
691/691 [==============================] - 6s 9ms/step - loss: 0.6379 - accuracy: 0.8181 - val_loss: 2.2516 - val_accuracy: 0.6824 - _timestamp: 1652349891.0000 - _runtime: 44.0000
Epoch 5/20
691/691 [==============================] - 6s 9ms/step - loss: 0.5379 - accuracy: 0.8459 - val_loss: 2.3045 - val_accuracy: 0.6790 - _timestamp: 1652349897.0000 - _runtime: 50.0000
Epoch 6/20
691/691 [==================

...

INFO:tensorflow:Assets written to: ./TrainedModels/LSTMeng2_hi_3_adam_20_0.2_32_256/assets


INFO:tensorflow:Assets written to: ./TrainedModels/LSTMeng2_hi_3_adam_20_0.2_32_256/assets


accuracy,▁▃▅▆▇▇▇█████
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▅▃▃▂▂▂▁▁▁▁▁
val_accuracy,▁▄▅▆▆▇█▇▇█▇█
val_loss,▁▂▁▂█▄▄▆▅▇▇▆
accuracy,0.91139
best_epoch,0
best_val_loss,2.20592
epoch,11
loss,0.28916
val_accuracy,0.71075


wandb: Agent Starting Run: yjpgasf3 with config:
wandb: 	batch_size: 32
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.3
wandb: 	epochs: 20
wandb: 	hidden: 128
wandb: 	latentDim: 256
wandb: 	numDecoders: 3
wandb: 	numEncoders: 2
wandb: 	optimiser: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 27)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, None, 256),  290816      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                                  
 input_2 (InputLayer)           [(None, None, 66)]   0           []                               
                                                                                              